In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, cast, avg, sum, count, count_distinct

In [2]:
spark = SparkSession.builder.appName("Challenge 2").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/02 12:46:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.format("csv").option("header", True).load("../resources/data.csv")

In [5]:
df.printSchema()
df.show(1)
df.count()

root
 |-- Row ID: string (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: string (nullable = true)

+------+--------------+----------+----------+------------+-----------+-------------+--------+-------------+---------+--------+-----------+------+--

9994

In [6]:
df = df.withColumn("Order Date", to_date(df["Order Date"])) \
        .withColumn("Sales", df["Sales"].cast("float")) \
        .withColumn("Quantity", df["Quantity"].cast("integer")) \
        .withColumn("Discount", df["Discount"].cast("float")) \
        .withColumn("Profit", df["Profit"].cast("float"))

In [7]:
df.groupBy("Country").agg(count("*").alias("Total Rows")
                          ,sum("Sales").alias("Total Sales")
                          ,sum("Quantity").alias("Total Quantity")
                          ,sum("Profit").alias("Total Profit")
                          ,count_distinct("Customer ID").alias("Total Stores")
                          ,count_distinct("Order ID").alias("Total Orders")
                          ,(sum("Sales") / count_distinct("Order ID")).alias("AOV")
                          ,(sum("Sales") / count_distinct("Customer ID")).alias("ASV")).show()

+-------------+----------+------------------+--------------+-----------------+------------+------------+-----------------+-----------------+
|      Country|Total Rows|       Total Sales|Total Quantity|     Total Profit|Total Stores|Total Orders|              AOV|              ASV|
+-------------+----------+------------------+--------------+-----------------+------------+------------+-----------------+-----------------+
|United States|      9994|2272449.8518802524|         57975|285707.6013599336|         793|        5009|453.6733583310546|2865.636635410154|
+-------------+----------+------------------+--------------+-----------------+------------+------------+-----------------+-----------------+

